In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
#Student
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [6]:
student_data_df.rename(columns={
    'ID_Student':'ID_Student__c',
    'StudentID':'StudentID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate' : 'BirthDate__c',
    'Gender': 'Gender__c',
}, inplace=True)
student_data_df.head()

,ID_Student__c,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [7]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'ID_Student__c': 33,
  'StudentID__c': '25004961',
  'LastName__c': 'Moore',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 34,
  'StudentID__c': '25003514',
  'LastName__c': 'Multak',
  'FirstName__c': 'Ilana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 35,
  'StudentID__c': '25005833',
  'LastName__c': 'Murillo',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 36,
  'StudentID__c': '25002589',
  'LastName__c': 'Romanowski',
  'FirstName__c': 'Kandra',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 37,
  'StudentID__c': '25007185',
  'LastName__c': 'Hoffer',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 38,
  'StudentID__c': '25006014',
  'LastName__c': 'Pooch

In [8]:
for rec in student_data_records:

    record = {
        'ID_Student__c': rec['ID_Student__c'],
        'StudentID__c': rec['StudentID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': rec['BirthDate__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

In [9]:
student_lookup_list = []

data = sf.query_all_iter("SELECT ID_Student__c, Name FROM Student__c")
for row in data:
    rec = {
        'ID_Student__c': row['ID_Student__c'],
        'StudentSF__c': row['Name']
    }
    student_lookup_list.append(rec)

In [10]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head(5)

,ID_Student__c,StudentSF__c
0,59.0,a0K3h000001qErA
1,57.0,a0K3h000001qEr0
2,52.0,a0K3h000001qEqb
3,40.0,a0K3h000001qEpd
4,44.0,a0K3h000001qEpx


In [11]:
class_lookup_list = []

data = sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class__c': row['ID_Class__c'],
        'ClassSF__c': row['Name']
    }
    class_lookup_list.append(rec)

In [12]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,ID_Class__c,ClassSF__c
0,4.0,a0H3h000003S9k4
1,1.0,a0H3h000003S9jp
2,2.0,a0H3h000003S9ju
3,3.0,a0H3h000003S9jz


In [13]:
#Classparticipant
query = '''
    SELECT
        cp.*
        
    FROM
        classparticipant cp
        
'''
classparticipant_data_df = pd.read_sql(query, conn)
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [14]:
classparticipant_data_df.rename(columns={
    'ID_ClassParticipant' : 'ID_ClassParticipant__c',
    'ID_Student': 'ID_Student__c',
    'ID_Class':'ID_Class__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c',
}, inplace=True)

classparticipant_data_df.head(5)

,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [15]:
classparticipant_data_df = pd.merge(classparticipant_data_df, student_lookup_df, how='left')
classparticipant_data_df.head()

,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c,StudentSF__c
0,1,33,1,2020-03-16,None,a0K3h000001qEp4
1,2,34,1,2020-03-16,None,a0K3h000001qEp9
2,3,35,1,2020-03-16,None,a0K3h000001qEpE
3,4,62,1,2020-03-16,None,a0K3h000001qErP
4,5,36,1,2020-03-16,None,a0K3h000001qEpJ


In [16]:
classparticipant_data_df = pd.merge(classparticipant_data_df, class_lookup_df, how='left')
classparticipant_data_df.head()

,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c,StudentSF__c,ClassSF__c
0,1,33,1,2020-03-16,None,a0K3h000001qEp4,a0H3h000003S9jp
1,2,34,1,2020-03-16,None,a0K3h000001qEp9,a0H3h000003S9jp
2,3,35,1,2020-03-16,None,a0K3h000001qEpE,a0H3h000003S9jp
3,4,62,1,2020-03-16,None,a0K3h000001qErP,a0H3h000003S9jp
4,5,36,1,2020-03-16,None,a0K3h000001qEpJ,a0H3h000003S9jp


In [17]:
classparticipant_data_records = classparticipant_data_df.to_dict(orient='records')
classparticipant_data_records

[{'ID_ClassParticipant__c': 1,
  'ID_Student__c': 33,
  'ID_Class__c': 1,
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'StudentSF__c': 'a0K3h000001qEp4',
  'ClassSF__c': 'a0H3h000003S9jp'},
 {'ID_ClassParticipant__c': 2,
  'ID_Student__c': 34,
  'ID_Class__c': 1,
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'StudentSF__c': 'a0K3h000001qEp9',
  'ClassSF__c': 'a0H3h000003S9jp'},
 {'ID_ClassParticipant__c': 3,
  'ID_Student__c': 35,
  'ID_Class__c': 1,
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'StudentSF__c': 'a0K3h000001qEpE',
  'ClassSF__c': 'a0H3h000003S9jp'},
 {'ID_ClassParticipant__c': 4,
  'ID_Student__c': 62,
  'ID_Class__c': 1,
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'StudentSF__c': 'a0K3h000001qErP',
  'ClassSF__c': 'a0H3h000003S9jp'},
 {'ID_ClassParticipant__c': 5,
  'ID_Student__c': 36,
  'ID_Class__c': 1,
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None

In [18]:
for rec in classparticipant_data_records:
 
    record = {
        'ID_ClassParticipant__c': rec['ID_ClassParticipant__c'],
        'ID_Student__c': rec['ID_Student__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'StartDate__c': rec['StartDate__c'].isoformat(),
        'EndDate__c': rec['EndDate__c'],
        'Student_Name__c': rec['StudentSF__c'],
        'Class_Name__c': rec['ClassSF__c']
    }
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)